# Data Science Capstone Project - Car Accident Severity

## Introduction / Business Problem

For my Capstone Project, I will explore severity of car accidents in Seattle from 2004 to present. Specifically, I would like to explore what type of attributes would most likely to cause collisions

## Data Section

I will use a sample data set "Collisions-All Years". 
In order to test the severity of collisions, I will use 5 attributes: INATTENTIONIND, WEATHER, ROADCOND, LIGHTCOND, SPEEDING.
